In [1]:
import pandas as pd

from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [2]:
fake_data_set=np.array([[4,5,1,2,3,1],[2,3,2,5,3,1],[1,5,5,2,1,1],[4,2,4,2,5,1],[1,3,3,5,3,1],[4,4,1,2,4,2]])


In [3]:
fake_data_set


array([[4, 5, 1, 2, 3, 1],
       [2, 3, 2, 5, 3, 1],
       [1, 5, 5, 2, 1, 1],
       [4, 2, 4, 2, 5, 1],
       [1, 3, 3, 5, 3, 1],
       [4, 4, 1, 2, 4, 2]])

In [4]:
matrix = pd.DataFrame(fake_data_set,index=['user1','user2','user3','user4','user5','user6'],columns=['Product1','Product2','Product3','Product4','Product5','Product6'])

In [5]:
matrix


,Product1,Product2,Product3,Product4,Product5,Product6
user1,4,5,1,2,3,1
user2,2,3,2,5,3,1
user3,1,5,5,2,1,1
user4,4,2,4,2,5,1
user5,1,3,3,5,3,1
user6,4,4,1,2,4,2


In [6]:
# standarisation par rapport a la moyenne = 0
def standarize(row):
    new_row=(row - row.mean())/(row.max()-row.min())
    return new_row

rating_std = matrix.apply(standarize)
rating_std

,Product1,Product2,Product3,Product4,Product5,Product6
user1,0.444444,0.444444,-0.416667,-0.333333,-0.041667,-0.166667
user2,-0.222222,-0.222222,-0.166667,0.666667,-0.041667,-0.166667
user3,-0.555556,0.444444,0.583333,-0.333333,-0.541667,-0.166667
user4,0.444444,-0.555556,0.333333,-0.333333,0.458333,-0.166667
user5,-0.555556,-0.222222,0.083333,0.666667,-0.041667,-0.166667
user6,0.444444,0.111111,-0.416667,-0.333333,0.208333,0.833333


In [7]:
# we are taking a transpose since we want similarity between items which need to be in rows
item_similarity = cosine_similarity(rating_std.T)
print(item_similarity)

[[ 1.         -0.07312724 -0.54232614 -0.60024505  0.73292627  0.43386092]
 [-0.07312724  1.         -0.16854997 -0.42640143 -0.70406806  0.13483997]
 [-0.54232614 -0.16854997  1.         -0.07905694 -0.33786231 -0.5       ]
 [-0.60024505 -0.42640143 -0.07905694  1.         -0.09712859 -0.31622777]
 [ 0.73292627 -0.70406806 -0.33786231 -0.09712859  1.          0.30714756]
 [ 0.43386092  0.13483997 -0.5        -0.31622777  0.30714756  1.        ]]


In [8]:
item_similarity_df = pd.DataFrame(item_similarity,index=['Product1','Product2','Product3','Product4','Product5','Product6'],columns=['Product1','Product2','Product3','Product4','Product5','Product6'])

In [9]:
item_similarity_df

,Product1,Product2,Product3,Product4,Product5,Product6
Product1,1.000000,-0.073127,-0.542326,-0.600245,0.732926,0.433861
Product2,-0.073127,1.000000,-0.168550,-0.426401,-0.704068,0.134840
Product3,-0.542326,-0.168550,1.000000,-0.079057,-0.337862,-0.500000
Product4,-0.600245,-0.426401,-0.079057,1.000000,-0.097129,-0.316228
Product5,0.732926,-0.704068,-0.337862,-0.097129,1.000000,0.307148
Product6,0.433861,0.134840,-0.500000,-0.316228,0.307148,1.000000


In [10]:
# get the similar product (make recommandaation)
def get_similar_Product(Product_name,user_rating):
    similar_score = item_similarity_df[Product_name]*(user_rating-2.5)
    similar_score = similar_score.sort_values(ascending=False)
    return similar_score
# i will give to Product1 4 so show me the produces that my i love it
Products = get_similar_Product('Product1',4)
# print(produit[products > 2.5 ])
Products

Product1    1.500000
Product5    1.099389
Product6    0.650791
Product2   -0.109691
Product3   -0.813489
Product4   -0.900368
Name: Product1, dtype: float64

In [11]:
# make recommandation for this user 
user_action = [('Product1',5),('Product5',1),('Product3',1)]  
similar_products =  pd.DataFrame()
for product,rating in user_action:
    similar_products = similar_products.append(get_similar_Product(product,rating),ignore_index=True)

similar_products.head()   
similar_products.sum().sort_values(ascending=False)

Product1    2.214100
Product6    1.373931
Product2    1.126109
Product5    0.839109
Product4   -1.236334
Product3   -2.349022
dtype: float64